In [1]:
import requests
import re
from bs4 import BeautifulSoup
import bs4
import traceback
import json
import pandas as pd
from requests.exceptions import RequestException
import time
import sys
import random

In [2]:
class Spider:#常用的爬取方法的简单封装
    def __init__(self,url):
        self.url=url

    def getHTML(self):#获取html的对应代码
        s = requests.session()
        s.keep_alive = False
        cookie = '''finger=1571944565; DedeUserID__ckMd5=7006860c5a98af8c; _uuid=9DAE78DC-5E52-F466-317B-F2F368AB791422981infoc; sid=5e6cr02r; DedeUserID=438645542; rpdid=|(ku|uY)YkJu0J'ul~l)m~ml|; bili_jct=a614670d8af9890b6bd2f1600f8a55e5; laboratory=1-1; SESSDATA=13b8f50b%2C1602559771%2Cb66bc*41; LIVE_BUVID=AUTO4015758979496252; CURRENT_QUALITY=0; CURRENT_FNVAL=80; blackside_state=1; bp_video_offset_438645542=438235222819008202; PVID=1; buvid3=60AEE8B4-6153-45C8-A114-D34A6CB1750D138389infoc'''
        agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36 Edg/85.0.564.68'
        headers={'User-Agent': agent, 'Cookie':cookie}
        try:
            #response=requests.get(url=self.url,headers=headers,timeout=30, verify=False)
            response=requests.get(url=self.url,headers=headers,timeout=30)
            response.raise_for_status()
            response.encoding=response.apparent_encoding
            return response.text
        except:
            time.sleep(5)
            return "网页访问失败"
        
        
    def setSoup(self):#获取soup对象
        html=self.getHTML()
        self.soup=BeautifulSoup(html,'html.parser')
    
    def findTag(self,tagName):#按照标签名查找标签
        return self.soup.find_all(tagName)
    
    def findTagByAttrs(self,tagName,attrs):
        return self.soup.find_all(tagName,attrs)

    
    def getBeautifyHTML(self):
        return self.soup.prettify()

In [3]:
def getPage(url):#爬取排行榜页面,核心代码
    spider=Spider(url)
    spider.setSoup()
    itemList=spider.findTagByAttrs('li','rank-item')
    pageContentList=[]
    for item in itemList:
        pageContentItem=[]
        for title in item.find_all('a','title'):
            pageContentItem.append(title.string)
            pageContentItem.append(title.attrs['href'])
            # print(title.string)
            
        for playnum in item.find_all('span','data-box'):
            pattern=r">([^<]+)<"
            n=re.findall(pattern,playnum.__str__())[0]
            pageContentItem.append(n)
            # print(n)
        
        for author in item.find_all('i', 'b-icon author'):
            parent = author.parent.parent.attrs['href']
            pageContentItem.append(parent)   
        # print(item.find_all('div','pts')[0].div.string)
        pageContentItem.append(item.find_all('div','pts')[0].div.string)
        pageContentList.append(pageContentItem)
    return pageContentList

In [4]:
def getURLFormBilibili():# 定义要获取的榜单
    date={
        #1:'日排行',
        #3:'三日排行',
        #7:'周排行',
        30:'月排行'
    }
    areatype={
        #0:'全站',
        #1:'动画',
        #168:'国创相关',
        #3:'音乐',
        #129:'舞蹈',
        #4:'游戏',
        #36:'知识',
        #188:'数码',
        #160:'生活',
        211:'美食'
        #119:'鬼畜',
        #155:'时尚',
        #5:'娱乐',
        #181:'影视'
    }
    ranktype={
       'all':'全站',
       #'origin':'原创',
        #'rookie':'新人'
    }
    submit={
        '0':'全部投稿',
        #'1':'近期投稿'
    }
    urlDict={}#存放相应url的字典
    for ranktypeItem in ranktype.keys():
        for areatypeItem in areatype.keys():
            for submitItem in submit.keys():
                for dateTypeItem in date.keys():
                    if((ranktypeItem=='rookie') and (areatypeItem==168)):
                        continue
                    title=ranktype[ranktypeItem]+'_'+areatype[areatypeItem]+'_'+submit[submitItem]+'_'+date[dateTypeItem]
                    destinaTionUrl='https://www.bilibili.com/ranking/{}/{}/{}/{}'.format(ranktypeItem,areatypeItem,submitItem,dateTypeItem)
                    urlDict[title]=destinaTionUrl
    return urlDict

In [5]:
# 获取弹幕
def getBullets(video_url):
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.103 Safari/537.36'}
    r = requests.get(video_url, headers=headers)
    r.encoding = r.apparent_encoding
    match_rule = r'cid=(.*?)&aid'
    if(re.search(match_rule,r.text)!=None):
        oid = re.search(match_rule,r.text).group().replace('cid=','').replace('&aid','')
    else:
        print("can't get bullets of "+video_url)
        oid = "1"
    xml_url = 'https://api.bilibili.com/x/v1/dm/list.so?oid='+oid
    #rint(xml_url)
    r_bullet = requests.get(xml_url, headers=headers)
    r_bullet.encoding = 'utf-8'
    html = r_bullet.text
    soup = BeautifulSoup(html, "html.parser")
    bullets = soup.find_all("d")
    bullet_dict = {}
    for bullet in bullets:
        bullet_dict[bullet.text] = bullet.attrs['p']
    return bullet_dict

In [6]:
def getAID(video_url):
    bvid = video_url.split("https://www.bilibili.com/video/",1)[1]
    if(bvid[:2]=="av"):
        return bvid.split("av",1)[1]
    else:
        headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.103 Safari/537.36'}
        response=requests.get(url=video_url,headers=headers,timeout=20)
        response.raise_for_status()
        response.encoding=response.apparent_encoding
        html = response.text
        aid = re.findall(r".*__INITIAL_STATE__={\"aid\":(.*),\"bvid\":\""+bvid, html)
        if(len(aid)==1):
            return aid[0]
        else:
            return "1"

In [7]:
def getVideoInfo(video_url):
    videoInfoDict = {}
    
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.103 Safari/537.36'}
    response=requests.get(url=video_url,headers=headers,timeout=20)
    response.raise_for_status()
    response.encoding=response.apparent_encoding
    html = response.text
    soup=BeautifulSoup(html,'html.parser')
    
    #get cats of the video
    catList = []
    if(soup.find("span", "a-crumbs")!=None):
        for a in soup.find("span", "a-crumbs"):
            if (a.name == "a"):
                catList.append(a.text)
        videoInfoDict["cats"] = catList
        #get the publish time
        videoInfoDict["pubtime"] = soup.find("span", "a-crumbs").next_sibling.text
    else:
        videoInfoDict["cats"] = {}
        videoInfoDict["pubtime"] = "none"
    
    
    #get the tags
    tagList = []
    if(soup.find("ul", "tag-area")!=None):
        for tag in soup.find("ul", "tag-area").children:
            if isinstance(tag, bs4.element.Tag):
                tagList.append(tag.text)
        videoInfoDict["tags"] = tagList
    else:
        videoInfoDict["tags"] = []
    
    aid = getAID(video_url)
    if(aid != "1"):
        video_info_url = "https://api.bilibili.com/x/web-interface/view?aid="+aid
        response1 =requests.get(url=video_info_url,timeout=20)
        response1.encoding=response1.apparent_encoding
        html = response1.text
        if response1.status_code == 200:
            j1 = json.loads(html)
            if("data" in j1):
                j1 = j1["data"]
                videoInfoDict['title'] = j1['title']
                videoInfoDict['view'] = j1['stat']['view']  # 播放量,视频没有播放量时显示会‘--’,不是整数，会抓取失败
                videoInfoDict['danmaku'] = j1['stat']['danmaku']  # 弹幕
                videoInfoDict['favorite'] = j1['stat']['favorite']  # 收藏
                videoInfoDict['coin'] = j1['stat']['coin']  # 硬币
                videoInfoDict['share'] = j1['stat']['share']  # 分享
                videoInfoDict['reply'] = j1['stat']['reply']  # 评论
                videoInfoDict['like'] = j1['stat']['share']  # 点赞
                videoInfoDict['duration'] = j1['duration']  # 时长
                videoInfoDict['cid'] = j1['cid']  # cid
                videoInfoDict['aid'] = j1['aid']  # aid
                videoInfoDict['desc'] = j1['desc']  # aid
            else:
                print("can't find video info of "+video_url)
                videoInfoDict['title'] = "none"
                videoInfoDict['view'] = "none"
                videoInfoDict['danmaku'] = "none"
                videoInfoDict['favorite'] = "none"
                videoInfoDict['coin'] = "none"
                videoInfoDict['share'] = "none"
                videoInfoDict['reply'] = "none"
                videoInfoDict['like'] = "none"
                videoInfoDict['duration'] = "none"
                videoInfoDict['cid'] = "none"
                videoInfoDict['aid'] = "none"
                videoInfoDict['desc'] = "none"
    else:
        print("can't find aid of "+video_url)
        videoInfoDict['title'] = "none"
        videoInfoDict['view'] = "none"
        videoInfoDict['danmaku'] = "none"
        videoInfoDict['favorite'] = "none"
        videoInfoDict['coin'] = "none"
        videoInfoDict['share'] = "none"
        videoInfoDict['reply'] = "none"
        videoInfoDict['like'] = "none"
        videoInfoDict['duration'] = "none"
        videoInfoDict['cid'] = "none"
        videoInfoDict['aid'] = "none"
        videoInfoDict['desc'] = "none"
    
    return videoInfoDict

In [8]:
def getAuthorInfo(author_url):
    mid = author_url.split("space.bilibili.com/",1)[1]
    user_info_url = "https://api.bilibili.com/x/space/acc/info?mid="+mid
    follow_info_url = "https://api.bilibili.com/x/relation/stat?vmid="+mid
    stat_url = "https://api.bilibili.com/x/space/upstat?mid="+mid
    video_url = "https://api.bilibili.com/x/space/arc/search?mid="+mid+"&ps=100&tid=0&pn=1&keyword=&order=pubdate&jsonp=jsonp"

    author_info_dict = {}

    spider_0 = Spider(user_info_url)
    spider_1 = Spider(follow_info_url)
    spider_2 = Spider(stat_url)
    spider_4 = Spider(video_url)

    html_0 = spider_0.getHTML()
    user_info_dict = json.loads(html_0)
    html_1 = spider_1.getHTML()
    follow_info_dict = json.loads(html_1)
    html_2 = spider_2.getHTML()
    #print(html_2)
    stat_dict = json.loads(html_2)
    html_4 = spider_4.getHTML()
    video_info_dict = json.loads(html_4)
    
    #print(stat_url)
    #print(html_2)

    author_info_dict["name"] = user_info_dict["data"]["name"]
    author_info_dict["sex"] = user_info_dict["data"]["sex"]
    author_info_dict["mid"] = user_info_dict["data"]["mid"]
    author_info_dict["following"] = follow_info_dict["data"]["following"]
    author_info_dict["follower"] = follow_info_dict["data"]["follower"]
    author_info_dict["likes"] = stat_dict["data"]["likes"]
    author_info_dict["article_view"] = stat_dict["data"]["article"]["view"]
    author_info_dict["video_view"] = stat_dict["data"]["archive"]["view"]
    author_info_dict["cat_dict"] = video_info_dict["data"]["list"]["tlist"]
    author_info_dict["video_list"] = video_info_dict["data"]["list"]["vlist"]
    
    return author_info_dict

In [9]:
class ShowProcess():
    """
    显示处理进度的类
    调用该类相关函数即可实现处理进度的显示
    """
    i = 0 # 当前的处理进度
    max_steps = 0 # 总共需要处理的次数
    max_arrow = 50 #进度条的长度
    infoDone = 'done'

    # 初始化函数，需要知道总共的处理次数
    def __init__(self, max_steps, infoDone = 'Done'):
        self.max_steps = max_steps
        self.i = 0
        self.infoDone = infoDone

    # 显示函数，根据当前的处理进度i显示进度
    # 效果为[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
    def show_process(self, i=None):
        if i is not None:
            self.i = i
        else:
            self.i += 1
        num_arrow = int(self.i * self.max_arrow / self.max_steps) #计算显示多少个'>'
        num_line = self.max_arrow - num_arrow #计算显示多少个'-'
        percent = self.i * 100.0 / self.max_steps #计算完成进度，格式为xx.xx%
        process_bar = '[' + '>' * num_arrow + '-' * num_line + ']'\
                      + '%.2f' % percent + '%' + '\r' #带输出的字符串，'\r'表示不换行回到最左边
        sys.stdout.write(process_bar) #这两句打印字符到终端
        sys.stdout.flush()
        if self.i >= self.max_steps:
            self.close()

    def close(self):
        print('')
        print(self.infoDone)
        self.i = 0

In [10]:
urlDict = getURLFormBilibili()

#process_bar = ShowProcess(len(urlDict), 'OK')
for urlName in urlDict:
    url = urlDict[urlName]
    tableName = urlName
    pageContentList = getPage(url)
    rank = 1
    rankList = []
    titleList = []
    urlList = []
    playList = []
    bulletList = []
    authorList = []
    authorLinkList = []
    scoreList = []
    
    for contentItem in pageContentList:
        rankList.append(rank)
        titleList.append(contentItem[0])
        urlList.append(contentItem[1])
        playList.append(contentItem[2])
        bulletList.append(contentItem[3])
        authorList.append(contentItem[4])
        authorLinkList.append(contentItem[5])
        scoreList.append(contentItem[6])
        rank += 1

    df = pd.DataFrame()
    df["排名"] = rankList
    df["标题"] = titleList
    df["链接"] = urlList
    df["播放量"] = playList
    df["弹幕数"] = bulletList
    df["up主"] = authorList
    df["up主链接"] = authorLinkList
    df["得分"] = scoreList
    df.to_excel('data\\'+tableName+".xlsx",encoding = 'utf-8')
    
    #获取每个视频的详细信息和弹幕
    url_bullet_dict = {}
    url_info_dict = {}
    for video_url in urlList:
        #print("getting bullets of "+video_url)
        bullet_dict = getBullets(video_url)
        #print("getting info of "+video_url)
        videoInfoDict = getVideoInfo(video_url)
        url_bullet_dict[video_url] = bullet_dict
        url_info_dict[video_url] = videoInfoDict

    with open('data\\'+tableName+'弹幕.json', 'w', encoding = 'utf-8') as fp:
        json.dump(url_bullet_dict, fp)
    with open('data\\'+tableName+'详情.json', 'w', encoding = 'utf-8') as fp:
        json.dump(url_info_dict, fp)
    
    #获取每个视频的up主信息
    url_authorinfo_dict = {}
    for author_url in authorLinkList:
        author_info_dict = getAuthorInfo(author_url)
        url_authorinfo_dict[author_url] = author_info_dict
    
    with open('data\\'+tableName+'up主.json', 'w', encoding = 'utf-8') as fp:
        json.dump(url_authorinfo_dict, fp)
    
    #process_bar.show_process()

D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverifi

D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverifi

D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverifi

D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverifi

D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverifi

...

D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverifi

D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverifi

D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverifi

D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverifi

D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bilibili.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverifi